In [ ]:
import heapq
import random

def heuristic(a, b):
    return abs(a[0] - b[0]) + abs(a[1] - b[1])

def is_valid_move(grid, x, y, obstacles, dynamic_agents, time_step):
    if 0 <= x < len(grid) and 0 <= y < len(grid[0]) and grid[x][y] != 'X':
        if (x, y) in obstacles:
            return False
        for agent in dynamic_agents:
            if time_step % (2 * len(agent)) < len(agent) and agent[time_step % len(agent)] == (x, y):
                return False
        return True
    return False

def a_star(grid, start, goal, obstacles, dynamic_agents):
    moves = [(0, 1), (1, 0), (0, -1), (-1, 0)]
    open_set = []
    heapq.heappush(open_set, (0, start, []))
    visited = set()
    
    while open_set:
        cost, (x, y), path = heapq.heappop(open_set)
        if (x, y) == goal:
            return path + [(x, y)], len(path)
        
        time_step = len(path)  # Ensure time step is correctly tracked
        if (x, y, time_step) in visited:
            continue
        visited.add((x, y, time_step))
        
        for dx, dy in moves:
            nx, ny = x + dx, y + dy
            if is_valid_move(grid, nx, ny, obstacles, dynamic_agents, time_step + 1):
                heapq.heappush(open_set, (cost + 1 + heuristic((nx, ny), goal), (nx, ny), path + [(x, y)]))
    
    return [], float('inf')

def resolve_collision(robot_paths):
    positions = {}
    for i, path in enumerate(robot_paths):
        for t, pos in enumerate(path):
            if (t, pos) in positions:
                positions[(t, pos)].append(i)
            else:
                positions[(t, pos)] = [i]
    
    for (t, pos), robots in positions.items():
        if len(robots) > 1:
            for r in robots:
                random.shuffle(robot_paths[r])
    
    return robot_paths

def find_paths(grid, robots, goals, dynamic_agents):
    robot_paths = []
    obstacles = set()
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'X':
                obstacles.add((i, j))
    
    for i, (start, goal) in enumerate(zip(robots, goals)):
        path, time = a_star(grid, start, goal, obstacles, dynamic_agents)
        robot_paths.append((path, time))
    
    resolved_paths = resolve_collision([p[0] for p in robot_paths])
    return [(path, len(path) - 1) for path in resolved_paths]

# Example Usage
grid = [
    ['S', '.', '.', '.', '.'],
    ['.', 'X', '.', '.', '.'],
    ['.', '.', '.', 'X', '.'],
    ['.', '.', '.', '.', '.'],
    ['.', '.', '.', '.', 'G']
]

dynamic_agents = [
    [(1, 1), (1, 2), (1, 3)],
    [(3, 3), (2, 3), (1, 3)]
]

robots = [(0, 0), (4, 0)]
goals = [(4, 4), (0, 4)]

paths = find_paths(grid, robots, goals, dynamic_agents)
for i, (path, time) in enumerate(paths):
    print(f"Robot {i+1} Path: {path}")
    print(f"Robot {i+1} Total Time: {time}")

Robot 1 Path: [(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (1, 4), (2, 4), (3, 4), (4, 4)]
Robot 1 Total Time: 8
Robot 2 Path: [(4, 0), (3, 0), (2, 0), (1, 0), (0, 0), (0, 1), (0, 2), (0, 3), (0, 4)]
Robot 2 Total Time: 8


In [ ]:
"""
Hafsa Imtiaz
22i - 0959
Section H
"""
import ast
import heapq

class Robot:
    def __init__(self, robot_id, start, goal, grid):
        self.id = f"R{robot_id}"
        self.start = start
        self.goal = goal
        self.current_position = start
        self.waqt = 0     
        self.safar = [start]    
        self.path = self.a_star(start, grid, {})  
    
    def heuristic(self, shuro, manzil):
        return abs(manzil[0] - shuro[0]) + abs(manzil[1] - shuro[1])
    
    def mere_humsaey(self, jagah, grid, agents):
        neighbours = []
        for dx, dy in [(-1,0), (1,0), (0,-1), (0,1)]:
            neighbor = (jagah[0] + dx, jagah[1] + dy)
            
            if not (0 <= neighbor[0] < len(grid) and 0 <= neighbor[1] < len(grid[0])):
                continue 
            if grid[neighbor[0]][neighbor[1]] == 'X' or any(agent_info['Path'][agent_info['Index']] == neighbor for agent_info in agents.values()):
                continue  

            neighbours.append(neighbor)
        
        return neighbours

    def a_star(self, shuro, grid, agents): # i'm a starrr
        pahar_heap = []
        heapq.heappush(pahar_heap, (0, shuro))  # (cost, node)
        came_from = {}  # this will store kon kaha se aya takay end pr path reconstruction
        g_n = {shuro: 0} # stores the g(n) values ---> (node: g(n))
        f_n = {shuro: 0 + self.heuristic(shuro, self.goal)} # stores the f(n) = g(n) + h(n) values ---> (node: h(n))

        while pahar_heap:
            _, current = heapq.heappop(pahar_heap)
            
            if current == self.goal:
                path = []
                while current in came_from:
                    path.append(current)
                    current = came_from[current]
                path.reverse()
                return path

            # get all valid neighbours
            neighbours = self.mere_humsaey(current, grid, agents)

            for neigh in neighbours:
                actual_cost = g_n[current] + 1  # new_g = g(current) + 1
                if neigh not in g_n or actual_cost < g_n[neigh]:
                    came_from[neigh] = current  # set current node as the parent node for the "neighbour" node takay end pr path making 
                    g_n[neigh] = actual_cost    
                    f_n[neigh] = actual_cost + self.heuristic(neigh, self.goal) # f(n) = g(n) + h(n)
                    heapq.heappush(pahar_heap, (f_n[neigh], neigh))
        return [] # koi rasta nahi

    def move(self, grid, agents, occupied_cells):
        if self.current_position == self.goal:
            return
        
        # no rasta??? 
        if not self.path: 
            self.path = self.a_star(self.current_position, grid, agents)

        # naya rasta ---> agents or collision
        while self.path and (self.path[0] in occupied_cells or any(agent_info['Path'][agent_info['Index']] == self.path[0] for agent_info in agents.values())):
            self.path = self.a_star(self.current_position, grid, agents)
        
        if self.path:
            pichla_x, pichla_y = self.current_position
            grid[pichla_x][pichla_y] = ' '

            self.current_position = self.path.pop(0)
            x, y = self.current_position
            grid[x][y] = self.id

            self.waqt += 1
            self.safar.append(self.current_position)
            occupied_cells.add(self.current_position)


def setup(grid_file, agents_file, robots_file):
    grid = []
    with open(grid_file, 'r') as file:
        N = int(file.readline().strip())
        for i in range(N):
            row = list(file.readline().replace("\n", ""))  
            if i == 0: M = len(row)
            #grid.append(row[::2]) # agar only even spaces count hongi
            grid.append(row)

    agents = {}
    with open(agents_file, 'r') as file:
        lines = file.readlines()
        i = 1
        for line in lines:
            agent_data = line.strip().split(' at times ')
            path = ast.literal_eval(agent_data[0].split(':')[1].strip())[0]
            times = [int(x) for x in ast.literal_eval(agent_data[1].strip())]
            sorted_path = [location for _, location in sorted(zip(times, path), key=lambda x: x[0])]
            agents[i] = {'Path': sorted_path, 'Index': 0}
            i += 1

    robots = []
    with open(robots_file, 'r') as file:
        lines = file.readlines()
        for i, line in enumerate(lines, start=1):
            start, goal = map(ast.literal_eval, line.strip().split(': Start ')[1].split(' End '))
            robot = Robot(i, start, goal, grid)
            robots.append(robot)
            x, y = goal
            grid[x][y] = f"G{i}"

    return (N, M), grid, agents, robots

def move_agents(agents, grid):
    for agent_id, agent_info in agents.items():
        path = agent_info['Path']
        index = agent_info.get('Index') 

        prev_position = path[index - 1]
        if prev_position:
            px, py = prev_position
            if grid[px][py] == f"A{agent_id}":
                grid[px][py] = ' ' 
        
        x, y = path[index]
        if grid[x][y] != 'X':
            #agent_info, index = change_agent_index(agent_info, index, path)
            grid[x][y] = f"A{agent_id}"
        change_agent_index(agent_info, index, path)

def change_agent_index(agent_info, index, path):
    if index == len(path) - 1:
        agent_info['Path'].reverse()
        agent_info['Index'] = 1  
    else:
        agent_info['Index'] += 1
    return agent_info, agent_info['Index'] 

def move_robots(robots, grid, agents):
    occupied_cells = {robot.current_position for robot in robots}  # kon kaha hai
    for robot in robots:
        occupied_cells.remove(robot.current_position) 
        robot.move(grid, agents, occupied_cells)
        occupied_cells.add(robot.current_position) 

def print_grid(grid):
    for row in grid:
        print(" ".join(row))

# Main execution
datasets = []
choice = input("Do you want to use hardcoded file paths? (y/n): ").lower()
if choice == "y" or choice == "yes":
    datasets = [
        ("data0.txt", "Agent0.txt", "Robots0.txt"),
        ("data1.txt", "Agent1.txt", "Robots1.txt"),
        ("data2.txt", "Agent2.txt", "Robots2.txt"),
        ("data3.txt", "Agnet3.txt", "Robots3.txt"),
        ("data4.txt", "Agent4.txt", "Robots4.txt"),
    ]
else:
    map_file = input("Enter the path for the map file: ")
    agent_file = input("Enter the path for the agent file: ")
    robot_file = input("Enter the path for the robots file: ")
    datasets = [(map_file, agent_file, robot_file)]

for map_file, agent_file, robot_file in datasets:
    print(f"\nRunning for {map_file}, {agent_file}, {robot_file}:")
    map_size, grid, agents, robots = setup(map_file, agent_file, robot_file)
    time = 0
    while not all(robot.current_position == robot.goal for robot in robots):
        move_agents(agents, grid)
        move_robots(robots, grid, agents)
        #print(f"Time: {time}")
        #print_grid(grid)
        #print("________________________________________________")
        time += 1

    print("\nAll Robots have reached their goal:")
    for robot in robots:
        print(f"Robot {robot.id[1:]}:\nStart: {robot.start} ---> Goal: {robot.goal}")
        print(f"Time: {robot.waqt}")
        print(f"Path: {robot.safar}\n")
